<center>
<h1>CSC-343 ARTIFICIAL INTELLIGENCE</h1>
<h1>PROGRAMMING ASSIGNMENT 7</h1>
<h1>HIDDEN MARKOV MODELS</h1>
</center>

<!-- <center><a href="https://simple.wikipedia.org/wiki/Buffalo_buffalo_Buffalo_buffalo_buffalo_buffalo_Buffalo_buffalo">
<img width="60%" src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/2c/Buffalo_sentence_1_parse_tree.svg/2880px-Buffalo_sentence_1_parse_tree.svg.png">
</a>


S = sentence,
NP = noun phrase,
RC = relative clause,
VP = verb phrase,
PN = proper noun,
N = noun,
V = verb
</center> -->
<br/><hr/><br/>

<center>
<a href="https://xkcd.com/1443/">
<img width="30%" src="https://imgs.xkcd.com/comics/language_nerd_2x.png">
</a>

</center>

<br/>
<hr/>


# Download Parts-of-Speech Data

* There are 3914 sentences in the dataset.


* Each sentence is an _in-order_ list of tuples: (token, tag).


* All sentences together contain a total of 100676 tuples. 


* The list of POS tags in the data is:

`{('.', 'punctuation'), \
 ('ADJ', 'adjective'), \
 ('ADP', 'adposition'), \
 ('ADV', 'adverb'), \
 ('CONJ', 'conjunction'), \
 ('DET', 'determiner'), \
 ('NOUN', 'noun'), \
 ('NUM', 'number'), \
 ('PRON', 'pronoun'), \
 ('PRT', 'particle'), \
 ('VERB', 'verb'), \
 ('X', 'other')}`

In [1]:
from tqdm import tqdm 
import pandas as pd 

def download_and_get_data():
  import nltk
  nltk.download(['treebank', 'universal_tagset'], quiet=True)
  pos_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
  return pos_data
 
data = download_and_get_data()

data[0]

[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET'),
 ('board', 'NOUN'),
 ('as', 'ADP'),
 ('a', 'DET'),
 ('nonexecutive', 'ADJ'),
 ('director', 'NOUN'),
 ('Nov.', 'NOUN'),
 ('29', 'NUM'),
 ('.', '.')]

In [2]:
len(data), sum([1 for sent in data for tup in sent])

(3914, 100676)

# Q1. Transition Probabilities

Compute the transition probabilities matrix from the data using the formula given below.

<br/>

$$P(t_{i} | t_{i-1}) = \frac{count(t_{i-1}, t_{t})}{count(t_{i-1})}$$
<br/>

Make sure you cater for start of sentences using a special `<START>` tag. 

You can do this either directly within the transition probabilities matrix (by having a relevant vector/column for it) _OR_ you can have a separate matrix/vector and call it the `initial` model. 

In [3]:
data[0]

tagSet = ['START', '.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB','X']

tagList = pd.DataFrame(0, index = tagSet, columns = tagSet)

for sent in data:
  for i, val in enumerate(sent):
    if i == 0:
      tagList.loc["START", val[1]] += 1
    else:
      tagList.loc[sent[i-1][1], val[1]] += 1

tagList

,START,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
START,0,320,161,505,205,201,927,1141,33,288,5,38,90
.,0,776,359,565,412,482,1114,1467,915,486,23,1003,229
ADJ,0,415,425,497,30,108,31,4474,133,4,69,77,134
ADP,0,392,1050,167,133,8,3194,3173,618,679,14,82,342
ADV,0,432,411,376,252,22,218,100,100,48,45,1093,73
CONJ,0,80,266,120,124,1,270,792,94,133,11,355,19
DET,0,154,1788,81,110,4,48,5569,193,31,2,346,398
NOUN,0,6926,350,5100,490,1229,377,7623,273,135,1267,4240,837
NUM,0,414,118,124,10,48,11,1252,656,5,96,64,746
PRON,0,111,200,62,93,14,26,573,20,21,34,1329,254


In [4]:
tagList = tagList.apply(lambda x: x/x.sum(), axis=1) # divide cell number by column total (transition probabilities ?)
tagList

,START,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
START,0.0,0.081758,0.041134,0.129024,0.052376,0.051354,0.236842,0.291518,0.008431,0.073582,0.001277,0.009709,0.022994
.,0.0,0.099093,0.045843,0.072149,0.052611,0.061550,0.142255,0.187332,0.116843,0.062061,0.002937,0.128081,0.029243
ADJ,0.0,0.064874,0.066437,0.077693,0.004690,0.016883,0.004846,0.699390,0.020791,0.000625,0.010786,0.012037,0.020947
ADP,0.0,0.039789,0.106577,0.016951,0.013500,0.000812,0.324198,0.322067,0.062728,0.068920,0.001421,0.008323,0.034714
ADV,0.0,0.136278,0.129653,0.118612,0.079495,0.006940,0.068770,0.031546,0.031546,0.015142,0.014196,0.344795,0.023028
CONJ,0.0,0.035320,0.117439,0.052980,0.054746,0.000442,0.119205,0.349669,0.041501,0.058720,0.004857,0.156733,0.008389
DET,0.0,0.017652,0.204952,0.009285,0.012609,0.000459,0.005502,0.638354,0.022123,0.003553,0.000229,0.039661,0.045621
NOUN,0.0,0.240094,0.012133,0.176795,0.016986,0.042604,0.013069,0.264256,0.009464,0.004680,0.043921,0.146982,0.029015
NUM,0.0,0.116817,0.033296,0.034989,0.002822,0.013544,0.003104,0.353273,0.185102,0.001411,0.027088,0.018059,0.210497
PRON,0.0,0.040555,0.073073,0.022653,0.033979,0.005115,0.009499,0.209353,0.007307,0.007673,0.012422,0.485568,0.092802


# Q2. Emission Probabilities 

Compute the emissions probability matrix from the data using the formula below: 

<br/>

$$ P(w_i | t_i) = \frac{count(t_i, w_i)}{count(t_i)}$$

<br/>

$t$ stands for tag and $w$ stands for word or token. 

In [5]:
unique = []
for sentence in data:
  for word in sentence:
    unique.append(word[0])
unique = set(unique)

# uniquev2 = set(word[0] for sentence in data for word in sentence) # same thing but in one line

wordList = pd.DataFrame(0, index=tagSet, columns=unique)

for sentence in data:
  for word in sentence:
    wordList[word[0]][word[1]] += 1
wordList

##########################################
# vocab = sorted(set([tup[0] for sentence in data for tup in sentence]))

# em = pd.DataFrame(0, index=tagSet[1:], columns = vocab)

# for sentence in data:
#   for i in range(len(sentence) - 1):
#     em[sentence[i][0]][sentence[i][1]] += 1

# em = em.apply(lambda row: row / row.sum(), axis=1)

# em

,Icahn,1787,Criticism,Sure,agricultural,Oil,Coniston,remarkable,challenge,jugglers,...,transportation,Md.,Association,*T*-125,felony,Dobson,Palisades,Elsewhere,566.54,PORTING
START,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
.,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ADJ,0,0,0,0,3,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
ADP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ADV,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
CONJ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DET,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NOUN,2,0,1,0,0,5,2,0,3,1,...,4,1,21,0,2,1,1,0,0,0
NUM,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
PRON,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
wordList = wordList.apply(lambda x: x / x.sum(), axis=1).fillna(0) # divide cell number by column total
wordList = wordList[sorted(wordList.columns)]
wordList

,!,#,$,%,&,','','30s,'40s,'50s,...,your,yourself,youth,youthful,yttrium-containing,zero,zinc,zip,zone,zoomed
START,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
.,0.000512,0.001366,0.061289,0.000000,0.000000,0.000854,0.058387,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ADJ,0.000000,0.000000,0.000000,0.000156,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000156,0.000156,0.000000,0.000000,0.000000,0.000000,0.000000
ADP,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ADV,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CONJ,0.000000,0.000000,0.000000,0.000000,0.037528,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DET,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NOUN,0.000000,0.000000,0.000000,0.015416,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000104,0.000000,0.000000,0.000000,0.000035,0.000000,0.000069,0.000000
NUM,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000282,0.000282,0.000282,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000282,0.000000,0.000000,0.000000,0.000000
PRON,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.008403,0.001096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
# wordList.sum(axis = 0)

# Q1. Filtering (Forward algorithm)

<br/>

$$ \text{Forward}_{~1:t} = \mathbf{\alpha} \cdot \text{Emission}_{~t} \cdot \mathbf{\sum\limits_{x_{t\text{-}1}}} (~\text{Transition}_{~t\text{-}1~\rightarrow~t } \cdot \text{Forward}_{~1:t\text{-}1}~) $$

<br/>

$$ P(X_t~|~e_{1:t}) = \mathbf{\alpha} \cdot P(e_t~|~X_t) \cdot \mathbf{\sum\limits_{x_{t-1}}} (~P(X_t~|~x_{t-1}) \cdot P(x_{t-1}~|~e_{1:t-1})~) $$

Using the forward algorithm/equation above, compute the following terms: 

\

*     $P(t_2~|~w_1=\text{Pierre}, w_2 = \text{Vinken})$

\

*     $P(t_6~|~w_1=\text{Pierre}, w_2 = \text{Vinken}, w_3=\textbf{,}~, w_4=\text{61}, w_5=\text{years}, w_6= \text{old})$

\

*     $P(t_9~|~w_1=\text{Pierre}, w_2 = \text{Vinken}, w_3=\textbf{,}~, w_4=\text{61}, w_5=\text{years}, w_6= \text{old}, w_7=\textbf{,}~,w_8=\text{will}, w_9=\text{join})$

\
<font color='darkred'>I'll leave it up to you if you want to use the transition and emission matrices of the whole dataset or create new matrices for just the first sentence (to help with the written assignment).</font>

In [8]:
#transition -- tagList
#emission -- wordList

# emissions1 = wordList.multiply(tagList.loc["START"], axis=0)
# emissions1_norm = emissions1.div(emissions1.sum(axis=0), axis=1)

# tagList.loc["START"] #P(R0)

def forward(sentence):
  sent = sentence.split()
  forward = pd.DataFrame()
  
  for i in range(len(sent)):
    
    token = sent[i]

    if i == 0:
      prev = tagList.loc["START"] #initial model
    else:
      prev = forward[i - 1] #recursive term? 
    
    p_tag = tagList.apply(lambda x: prev * x).sum(axis = 0)
    
    p_word_given_tag = wordList[token]
    p_tag_given_word = p_tag * p_word_given_tag
    alpha =  1 / p_tag_given_word.sum()
    p_tag_given_word = p_tag_given_word * alpha 
    
    forward[i] = p_tag_given_word

  forward.columns = range(1, len(sent) + 1)
  return forward

forward('Pierre Vinken , 61 years old , will join')

,1,2,3,4,5,6,7,8,9
START,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
.,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000,0.0
ADJ,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
ADP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
ADV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
CONJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
DET,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
NOUN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.002448,0.0
NUM,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0
PRON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


# Q2. Prediction

<br/>

$$ \text{Prediction}_{~x_{t+1}} =\mathbf{\sum\limits_{x_{t}}} ~\text{Transition}_{~{t}\rightarrow{t+1}} \cdot \text{Forward}_{~1:t}~ $$

<br/>

$$ P(X_{t+1}~|~e_{1:t}) =\mathbf{\sum\limits_{x_{t}}} (~P(X_{t+1}~|~x_{t}) \cdot P(x_{t}~|~e_{1:t})~) $$

\
Using the equation for prediction above to compute the following terms: 

\

*     $P(t_3~|~w_1=\text{Pierre}, w_2 = \text{Vinken})$

\

*     $P(t_6~|~w_1=\text{Pierre}, w_2 = \text{Vinken}, w_3=\textbf{,}~, w_4=\text{61}, w_5=\text{years})$






In [9]:
# p_tag_given_word = forward[1]

# #p_t3 = tagList.multiply(p_tag_given_vinken, axis=0).sum(axis=0) # brycen
# p_t3 = tagList.apply(lambda x: x * p_tag_given_word).sum(axis=0) # sultan

# def prediction(sent):
#   sent = sent.split()
#   p_tag_given_word = forward[t+1]
#   p_tag = tagList.apply(lambda x: x * p_tag_given_word).sum(axis=0)
#   return p_tag

# prediction("Pierre Vinken , 61 years ")

def prediction(sentence):
  p_tag_given_word = forward(sentence)
  p_ti = tagList.apply(lambda x: x * p_tag_given_word.sum(axis=1)).sum(axis=0) #i have no idea what I did here
  p_ti = p_ti * (1 / p_ti.sum()) #alpha somehow
  return p_ti.fillna(0)

prediction("Pierre Vinken , 61 years ")

START    0.000000
.        0.187239
ADJ      0.023108
ADP      0.127504
ADV      0.021278
CONJ     0.040581
DET      0.036913
NOUN     0.266675
NUM      0.066067
PRON     0.015502
PRT      0.032358
VERB     0.117417
X        0.065357
dtype: float64

# Q3. Smoothing (Forward Backward Algorithm)

<br/>

<center><img width="75%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/forward_backward.png"></center>

<br/>

$ \text{Backward}_{~k+1:t  } = \mathbf{\sum\limits_{k+1}}~\text{Emission}_{~k+1} \cdot ~\text{Transition}_{~k\text{+}1~\rightarrow~k } \cdot \text{Backward}_{~k+2:t}~ $

<br/>

$ P(e_{k+1:t} | X_k) = \mathbf{\sum\limits_{x_{k+1}}}~P(e_{k+1} |~x_{k+1}) \cdot P(x_{k+1} |~X_k) \cdot P(e_{k+2:t} |~X_{k+1})~ $

<br/>

$\text{Forward}_{~1:k} = P(X_k~|~e_{1:k} )$

\

$\text{Backward}_{~k+1:t} = P(e_{k+1:t}~|~X_k) $

\

$P(X_k | e_{1:t} ) = \text{Forward}_{~1:k}~\times \text{Backward}_{~k+1:t} $

\

where $\times$ is pointwise multiplication of vectors.

\

Use the equations above to compute the following term: 

\

* $P(t_6~|~\mathbf{e}=\text{Pierre Vinken , 61 years old , will join})$

In [10]:
def smoothing(ev):
  backward = pd.DataFrame(0, index=tagSet, columns=range(1,len(ev)+1))
  ev2 = ev.split()

  for i in reversed(range(1,len(ev2)+1)):
      if i == len(ev2):
          backward[i] = 1
      else:
          backward[i] = tagList.apply(lambda x: x * wordList[ev2[i]] * backward[i+1], axis = 1).sum(axis = 1)

  sv= forward(ev) * backward
  fwd = forward(ev)
  sv = fwd.apply(lambda x: x * backward[x.name])
  sv = sv.apply(lambda x: x * (1/x.sum()))

  sv.columns = ev2
  return sv

smoothing('Pierre Vinken , 61 years old , will join')

,Pierre,Vinken,",",61,years,old,",",will,join
START,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
.,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.00000,0.0
ADJ,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00000,0.0
ADP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
ADV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
CONJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
DET,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
NOUN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.00213,0.0
NUM,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00000,0.0
PRON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0


# Q4. Most likely explanation (Viterbi Algorithm)

<br/>

$$ \text{argmax}_{x_{1:t}} P(\mathbf{x}_{1:t}~|~\mathbf{e}_{1:t}) =  P(e_{t+1}~|~X_{t+1}) \cdot~ \text{argmax}_{x_t} ~\mathbf{P}(X_{t+1}~|~\mathbf{x}_{t}) \cdot ~\text{argmax}_{x_{1:t-1}} P(\mathbf{x}_{1:t-1}~, \mathbf{x}_t, ~e_{1:t}) $$

<br/>

Using the Transition and Emission probabilities of the <u>entire dataset</u> along with the Viterbi algorithm, infer Part-of-Speech tags for <u>every token of every sentence in the data</u>. 

# Q5. Accuracy

Calculate the accuracy score of your inferred Part-of-Speech tags. Please note the accuracy score for the entire dataset would be calculated at the <u>token level, not at the sentence level</u>.

<hr/>


<center><i><h3> "Language isn't a formal system. Language is Glorious Chaos."</h3></i></center>
<br/>
<a href="https://xkcd.com/1576/">
<center><img width="65%" src="https://imgs.xkcd.com/comics/i_could_care_less_2x.png"></center>
</a>

<br/>
<hr/>